<a href="https://colab.research.google.com/github/rui1011/LLMATCH_RuiKomatsu/blob/main/LLMATCH_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%%capture

!pip install llama-index --upgrade
!pip install llama-index
!pip install llama-index-core
!pip install llama-index-embeddings-huggingface
!pip install torch
!pip install langchain-community
!pip install --upgrade llama-index
!pip install --upgrade llama-index python-dotenv pydantic PyYAML
!pip install llama-index-llms-huggingface
!pip install --upgrade llama-index-llms-huggingface

!pip uninstall -y llama-index-agent-openai llama-index-embeddings-openai \
               llama-index-llms-openai llama-index-multi-modal-llms-openai \
               llama-index-program-openai llama-index-question-gen-openai

!pip install llama-index-llms-langchain
!pip install langchain_community pypdf
!pip install -U bitsandbytes
!pip install -U "transformers>=4.48.0" sentence-transformers
!pip install llama-index-embeddings-huggingface
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers
!pip install autoawq
!pip install -q -U google-generativeai
!pip install pdf2image
!pip install pillow
!sudo apt-get install poppler-utils -y
!pip install git+https://github.com/huggingface/transformers
!pip install decord
!pip install -U sentence-transformers

In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import Document, QueryBundle
from llama_index.core.service_context import ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, AutoModelForSequenceClassification
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from llama_index.core import Settings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
import numpy as np
from huggingface_hub import HfFolder
from huggingface_hub import whoami
from llama_index.core.prompts import PromptTemplate
import textwrap
import os
from pathlib import Path
from google import genai
import PIL.Image
from pdf2image import convert_from_path
from llama_index.core import Document as LlamaIndexDocument, VectorStoreIndex
from langchain.schema import Document as LangChainDocument

In [4]:
from llama_index.core.schema import Document as LlamaDocument

In [5]:
def rerank_candidates_with_cross_encoder(query, candidates, model_name="hotchpotch/japanese-reranker-cross-encoder-xsmall-v1", device="cpu", threshold=0.7):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

  pairs = [[query, candidate] for candidate in candidates]
  inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors="pt").to(device)

  with torch.no_grad():
    outputs = model(**inputs)
    scores = torch.sigmoid(outputs.logits).squeeze(1).tolist()

  ranked_candidates = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
  ranked_candidates = [(candidate, score) for candidate, score in ranked_candidates if score >= threshold]
  return ranked_candidates

In [6]:
def query_engine(query, index, llm, top_k=5, threshold=0.7):
    import torch, textwrap
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    qe = index.as_query_engine(similarity_top_k=top_k)
    nodes = qe.query(QueryBundle(query_str=query)).source_nodes  # Node list

    candidates = [(n.text, n.metadata.get("page", "?")) for n in nodes]

    texts = [t for t, _ in candidates]
    ranked_scores = rerank_candidates_with_cross_encoder(query, texts,
                                                         device=device,
                                                         threshold=threshold)

    ranked_chunks = []
    for (txt, score) in ranked_scores:
        page = next(p for t, p in candidates if t == txt)
        ranked_chunks.append((txt, page, score))

    context = "\n\n".join([f"[p{pg}] {txt}" for txt, pg, _ in ranked_chunks])

    prompt = f"""
あなたは知識豊富な周南公立大学情報科学部情報科学科のアシスタントです。以下のルールに従って回答してください。

【思考手順】
- Let's think step by step.
- 最終回答は簡潔（100〜200字）でまとめる。
- 質問とコンテキストに矛盾がないか確認し、矛盾があればコンテキストを優先。

【出力ルール】
- 根拠ページを (pX) として文末に添える (例: …です。(p3))
- 無関係な情報は含めない。

【コンテキスト】
{context}

【質問】:{query}

【最終回答】:
"""

    return llm.generate([prompt])

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

In [8]:
!pip install huggingface_hub

from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

if HF_TOKEN:
    login(HF_TOKEN)
    print("Successfully logged in to Hugging Face!")
else:
    print("HF_TOKEN is not set. Please add it to the Colab secrets.")

Successfully logged in to Hugging Face!


In [9]:
model_id = "elyza/Llama-3-ELYZA-JP-8B-AWQ"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.bfloat16,
      )

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/900 [00:00<?, ?B/s]

`torch.bfloat16` is not supported for AWQ CUDA kernels yet. Casting to `torch.float16`.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

In [10]:
text_generation_pipeline = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 256,
    temperature = 0.8,
    do_sample=True,
)

Device set to use cuda:0


In [11]:
my_llm = HuggingFacePipeline(pipeline = text_generation_pipeline)
Settings.llm = my_llm

/tmp/ipython-input-11-3855559067.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  my_llm = HuggingFacePipeline(pipeline = text_generation_pipeline)


In [12]:
!pip show transformers

Name: transformers
Version: 4.54.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autoawq, llama-index-llms-huggingface, peft, sentence-transformers


In [13]:
embedding_model = HuggingFaceEmbedding(
    model_name="cl-nagoya/ruri-v3-310m",
    device="cuda",
    max_length = 2048,
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [14]:
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [ ]:
def extract_text(response):
    if response is None or not hasattr(response, 'candidates'):
        print("Error: Response is None or has no 'candidates' attribute")
        return ""
    extracted = ""
    for candidate in response.candidates:
        if not hasattr(candidate, 'content') or candidate.content is None:
            print("Error: Candidate has no 'content'")
            continue
        if not hasattr(candidate.content, 'parts') or candidate.content.parts is None:
            print("Error: Candidate content has no 'parts'")
            continue
        for part in candidate.content.parts:
            if hasattr(part, 'text'):
                extracted += part.text + "\n"
    return extracted.strip()

In [ ]:
pdf_path = Path("/content/drive/MyDrive/experimental_file/experimental_file.pdf")

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

pages = convert_from_path(str(pdf_path), dpi=300)

pages = pages[:16]

CHUNKING_PROMPT = """\
OCR the following page into Markdown. Tables should be formatted as HTML.
Do not surround your output with triple backticks.
Chunk the document into sections of roughly 250 - 1000 words.
Surround each chunk with <chunk> and </chunk> tags.
Preserve as much content as possible, including headings, tables, etc.
Don't try to output any image.
"""

documents = []

for i, page in enumerate(pages):
    print(f"Processing Page {i+1}")
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[CHUNKING_PROMPT, page]
        )
        print(f"API Response for Page {i+1}: {response}")
        ocr_text = extract_text(response)
        print(f"Raw OCR Text for Page {i+1}: '{ocr_text}'")
        document = Document(
            text=f"--- OCR Result for Page {i+1} ---\n{ocr_text}",
            metadata={"page": i+1}
        )
        documents.append(document)
        print(f"Document Text for Page {i+1}: '{document.text}'")
    except Exception as e:
        print(f"Error processing Page {i+1}: {e}")
        document = Document(
            text=f"--- OCR Result for Page {i+1} ---\n[Error: OCR Failed]",
            metadata={"page": i+1}
        )
        documents.append(document)

ストリーミング出力は最後の 5000 行に切り捨てられました。
<td></td>
<td>1</td>
<td>○</td>
<td></td>
<td></td>
</tr>
<tr><td></td>
<td></td>
<td>卒業論文</td>
<td>4通</td>
<td></td>
<td>◎</td>
<td></td>
<td></td>
<td></td>
</tr>
</tbody>
</table>
<p>◎…卒業必修科目
○…選択科目</p>
</chunk>'
Document Text for Page 5: '--- OCR Result for Page 5 ---
<chunk>
情報科学部情報科学科 授業科目表
2024年度入学 情報科学部情報科学科 授業科目表
</chunk>

<chunk>
<table>
<thead>
<tr>
<th>科目区分</th>
<th>授業科目の名称</th>
<th>配当年次</th>
<th>単位数区分</th>
<th>プログラム</th>
<th>卒業要件</th>
</tr>
<tr>
<th></th>
<th></th>
<th></th>
<th>必修</th>
<th>選択</th>
<th>DS</th>
<th>IE</th>
<th>BA</th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<td>総合科目</td>
<td>人間形成と個性伸張のための科目群</td>
<td>周南Well-being創生入門</td>
<td>1前</td>
<td></td>
<td>2</td>
<td>◎</td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td>持続可能な社会とダイバーシティ</td>
<td>1前</td>
<td></td>
<td>2</td>
<td>◎</td>
<td></td>
<td></td>
</tr>
<tr><td></td>
<td></td>
<td>教養スポーツ実習Ⅰ</td>
<td>1前</td>
<td></td>
<td>1</td>
<td>○</td>
<td></td>
<td></td>
</tr>
<tr><td>

In [ ]:
langchain_docs = [LangChainDocument(page_content=doc.text, metadata=doc.metadata) for doc in documents]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

docs = text_splitter.split_documents(langchain_docs)

In [ ]:
llama_docs = [LlamaDocument.from_langchain_format(d) for d in docs]

Settings.llm = my_llm
Settings.embed_model = embedding_model
index = VectorStoreIndex.from_documents(llama_docs, show_progress=True)

/usr/local/lib/python3.11/dist-packages/llama_index/core/indices/base.py:113: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)


Parsing nodes:   0%|          | 0/396 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/396 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id
generation_config = model.generation_config
generation_config.pad_token_id = tokenizer.eos_token_id

In [ ]:
query = """
情報科学部の特色を教えてください。
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


情報科学部の特色は、実学を重視したカリキュラムを通して、情報技術を基盤にした問題解決能力を養う点です。特に、データサイエ
ンスや人工知能、IoT、Cybersecurityなどの分野に力を入れており、実践的なスキルを身に付けることができます。
さらに、企業との協同研究やインターンシップなどの機会を提供し、学生が即戦力として社会に貢献できる人材を育成することを目指
しています。(p4)


In [ ]:
query = """
情報科学部はセメスター制とクオーター制の授業、どちらが多く実施されていますか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


情報科学部で開講される専門科目の多くはクオーター制で実施されています。


In [ ]:
query = """
情報科学部は1年を通じてセメスター制の授業の方が、クオーター制の授業より多く実施されていますか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


情報科学部では、セメスター制で開講される科目とクオーター制で開講される科目が混在しています。したがって、セメスター制の授
業がクオーター制の授業より多く実施されているとは一概に言えません。


In [ ]:
query = """
AIに関して学びたいのですが、情報科学部の3つのプログラムのうちどのプログラムが最適でしょうか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


AIを学ぶには、データサイエンスプログラムが最適です。データサイエンスプログラムは、データ分析や機械学習を通して、AIの
基礎を学ぶことができます。根拠は、情報科学部のプログラム紹介(p7)です。


In [ ]:
query = """
大規模なデータを分析してみたいのですが、それを学ぶには情報科学部の3つのプログラムのうちどのプログラムが最適でしょうか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


データサイエンス・プログラムが最適です。データサイエンス・プログラムでは、大量のデータをどのように分析・活用するかを学ぶ
ことができます。


In [ ]:
query = """
金融分野の動向をデータ分析を通じて研究したいのですが、情報科学科の3つのプログラムのうちどのプログラムが最適でしょうか？
"""
print("回答を生成しています。少々お待ちください。")
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


回答を生成しています。少々お待ちください。


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


情報科学科には「情報科学コース」「データサイエンスコース」「情報システムコース」がありますが、金融分野の動向をデータ分析
を通じて研究するには「データサイエンスコース」が最適です。データサイエンスコースでは、統計学や機械学習を用いてデータ分析
を行うことができます。金融分野の研究では、時系列データや大量のデータを分析する必要があり、データサイエンスコースのカリキ
ュラムは非常に有効です。(p5)


In [ ]:
query = """
4年次において、卒業論文の提出締め切りはいつ頃ですか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


周南公立大学の情報科学部情報科学科の卒業論文の提出締め切りは、4年次の「卒業論文中間発表会」後です。具体的には、例年12
月上旬に開催される「卒業論文中間発表会」が終了した後、指導教員と相談の上、個別に設定されます。詳細な日程は、指導教員や学
科事務室に確認する必要があります。


In [ ]:
query = """
情報科学部で取得できる教職課程の免許状は何ですか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


情報科学部では、情報科の教職課程の免許状が取得可能です。


In [ ]:
query = """
周南公立大学は日本のどこに位置しますか？
"""
response = query_engine(query, index, my_llm)
final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
wrapped_answer = textwrap.fill(final_answer, width=60)
print(wrapped_answer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


周南公立大学は、山口県下松市に位置します。


In [ ]:
def answer_question(query):
  print("回答を生成しています。少々お待ちください。")
  response = query_engine(query, index, my_llm)
  final_answer = response.generations[0][0].text.split("【最終回答】:")[-1].strip()
  wrapped_answer = textwrap.fill(final_answer, width=60)
  return wrapped_answer

In [ ]:
user_q = input("質問を入力してください: ")

print(answer_question(user_q))

質問を入力してください: 卒業要件にGPAは関係ありますか？
回答を生成しています。少々お待ちください。


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


周南公立大学の卒業要件には、GPAは関係します。具体的には、卒業要件の条件に「GPAが2.5以上であること」が明記されて
います。(p15)。したがって、学生はこの条件を満たす必要があります。
